***proposed model***

In [ ]:
import time
import random
import matplotlib.pyplot as plt

inverse_des_sbox = {v: k for k, v in enumerate(des_sbox)}

def des_sbox_substitution(value):
    byte1 = (value >> 8) & 0xFF
    byte2 = value & 0xFF
    return (des_sbox[byte1] << 8) | des_sbox[byte2]

def inverse_des_sbox_substitution(value):
    byte1 = (value >> 8) & 0xFF
    byte2 = value & 0xFF
    return (inverse_des_sbox[byte1] << 8) | inverse_des_sbox[byte2]

def des_permutation(value):
    permuted = 0
    permutation_table = [15, 6, 11, 3, 8, 0, 12, 4, 7, 14, 1, 10, 2, 9, 13, 5]
    for i, bit_pos in enumerate(permutation_table):
        if value & (1 << bit_pos):
            permuted |= (1 << i)
    return permuted

def inverse_des_permutation(value):
    inverse_permuted = 0
    permutation_table = [15, 6, 11, 3, 8, 0, 12, 4, 7, 14, 1, 10, 2, 9, 13, 5]
    for i, bit_pos in enumerate(permutation_table):
        if value & (1 << i):
            inverse_permuted |= (1 << bit_pos)
    return inverse_permuted

def des_key_mixing(value, round_key):
    return value ^ round_key

# Complete AES Algorithm
inv_aes_sbox = {v: k for k, v in enumerate(aes_sbox)}

def aes_substitute(byte):
    return aes_sbox[byte]

def inverse_aes_substitute(byte):
    return inv_aes_sbox[byte]

def aes_shift_rows(value):
    byte1 = (value >> 8) & 0xFF
    byte2 = value & 0xFF
    return (byte2 << 8) | byte1

def inverse_aes_shift_rows(value):
    byte1 = (value >> 8) & 0xFF
    byte2 = value & 0xFF
    return (byte1 << 8) | byte2

def gf_mult(a, b):
    p = 0
    for _ in range(8):
        if b & 1:
            p ^= a
        hi_bit_set = a & 0x80
        a <<= 1
        if hi_bit_set:
            a ^= 0x1B
        b >>= 1
    return p & 0xFF

def aes_mix_columns(value):
    byte1 = (value >> 8) & 0xFF
    byte2 = value & 0xFF
    mixed1 = gf_mult(byte1, 2) ^ gf_mult(byte2, 3)
    mixed2 = gf_mult(byte2, 2) ^ gf_mult(byte1, 3)
    return (mixed1 << 8) | mixed2

def inverse_aes_mix_columns(value):
    byte1 = (value >> 8) & 0xFF
    byte2 = value & 0xFF
    inv_mixed1 = (gf_mult(byte1, 0x0E) ^ gf_mult(byte2, 0x0B) ^
                  gf_mult(byte1, 0x0D) ^ gf_mult(byte2, 0x09))
    inv_mixed2 = (gf_mult(byte2, 0x0E) ^ gf_mult(byte1, 0x0B) ^
                  gf_mult(byte2, 0x0D) ^ gf_mult(byte1, 0x09))
    return ((inv_mixed1 & 0xFF) << 8) | (inv_mixed2 & 0xFF)

# Generate 12 round keys from a base key
def generate_round_keys(base_key, num_rounds=12):
    return [(base_key ^ (i * 0x9F37)) & 0xFFFF for i in range(num_rounds)]

# AES-DES Encryption
def aes_des_encrypt(data, round_keys, num_rounds=12):
    left, right = (data >> 16) & 0xFFFF, data & 0xFFFF
    print(f"Initial Data: {hex(data)}")

    for round_num in range(num_rounds):
        round_key = round_keys[round_num]

        left = aes_substitute(left & 0xFF) | (aes_substitute((left >> 8) & 0xFF) << 8)
        left = aes_shift_rows(left)
        left = aes_mix_columns(left)
        left ^= round_key

        right = des_sbox_substitution(right)
        right = des_permutation(right)
        right = des_key_mixing(right, round_key)

        left, right = right, left

        print(f"After Round {round_num + 1}: {hex((left << 16) | right)}")

    encrypted = (left << 16) | right
    print(f"Final Encrypted Data: {hex(encrypted)}\n")
    return encrypted

# AES-DES Decryption
def aes_des_decrypt(data, round_keys, num_rounds=12):
    left, right = (data >> 16) & 0xFFFF, data & 0xFFFF
    print(f"Encrypted Data: {hex(data)}")

    for round_num in reversed(range(num_rounds)):
        round_key = round_keys[round_num]

        left, right = right, left
        right = des_key_mixing(right, round_key)
        right = inverse_des_permutation(right)
        right = inverse_des_sbox_substitution(right)

        left ^= round_key
        left = inverse_aes_mix_columns(left)
        left = inverse_aes_shift_rows(left)
        left = inverse_aes_substitute(left & 0xFF) | (inverse_aes_substitute((left >> 8) & 0xFF) << 8)

        print(f"After Round {num_rounds - round_num}: {hex((left << 16) | right)}")

    decrypted = (left << 16) | right
    print(f"Final Decrypted Data: {hex(decrypted)}\n")
    return decrypted

# RSA Key Generation and Operations
def generate_rsa_keys():
    p, q = 104729, 99991
    n = p * q
    phi = (p - 1) * (q - 1)
    e = 65537
    d = mod_inverse(e, phi)
    return (n, e, d)

def mod_inverse(a, m):
    m0, x0, x1 = m, 0, 1
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    return x1 + m0 if x1 < 0 else x1

def rsa_encrypt(data, e, n):
    return pow(data, e, n)

def rsa_decrypt(data, d, n):
    return pow(data, d, n)



base_key = 0xA3F4B67C


# Performance Measurement
# Performance Measurement
def measure_performance(data):
    n, e, d = generate_rsa_keys()
    round_keys = generate_round_keys(base_key)

    # Measure AES-DES Encryption Time
    start_time = time.time()
    aes_des_encrypted = aes_des_encrypt(data, round_keys)
    aes_des_time = (time.time() - start_time) * 1000  # Convert to ms

    # Measure RSA Encryption Time
    start_time = time.time()
    rsa_encrypted = rsa_encrypt(aes_des_encrypted, e, n)
    rsa_time = (time.time() - start_time) * 1000

    # Total Encryption Time
    total_encryption_time = aes_des_time + rsa_time

    # Measure RSA Decryption Time
    start_time = time.time()
    rsa_decrypted = rsa_decrypt(rsa_encrypted, d, n)
    rsa_decrypt_time = (time.time() - start_time) * 1000

    # Measure AES-DES Decryption Time
    start_time = time.time()
    decrypted_data = aes_des_decrypt(rsa_decrypted, round_keys)
    aes_des_decrypt_time = (time.time() - start_time) * 1000

    # Total Decryption Time
    total_decryption_time = rsa_decrypt_time + aes_des_decrypt_time

    # Data Expansion Calculation
    original_size = data.bit_length()  # Original data size in bits
    encrypted_size = rsa_encrypted.bit_length()  # Encrypted data size in bits
    expansion_ratio = ((encrypted_size - original_size) / original_size) * 100 if original_size > 0 else 0  # Percentage increase

    return aes_des_time, rsa_time, rsa_decrypt_time, aes_des_decrypt_time, expansion_ratio


plaintext = 0x1269978  # Example 32-bit input
aes_des_time, rsa_time, rsa_decrypt_time, aes_des_decrypt_time, expansion_ratio = measure_performance(plaintext)

# Total Decryption Time
total_decryption_time = rsa_decrypt_time + aes_des_decrypt_time

# Total Encryption Time
total_encryption_time = aes_des_time + rsa_time

# Print the Results
print(f"Total Encryption Time: {total_encryption_time:.2f} ms")
print(f"Total Decryption Time: {total_decryption_time:.2f} ms")
print(f"Data Expansion Percentage: {expansion_ratio:.2f}%")

# Plot Performance vs Security Trade-off
labels = ['AES-DES Encrypt', 'RSA Encrypt', 'RSA Decrypt', 'AES-DES Decrypt', 'Data Expansion (%)']
values = [aes_des_time, rsa_time, rsa_decrypt_time, aes_des_decrypt_time, expansion_ratio]

plt.figure(figsize=(10, 5))
plt.bar(labels, values, color=['blue', 'red', 'green', 'purple', 'orange'])
plt.ylabel('Time (ms) / Expansion (%)')
plt.title('Performance vs. Security Trade-off')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show values on top of bars
for i, v in enumerate(values):
    plt.text(i, v + 0.5, f"{v:.2f}", ha='center', fontsize=10)

plt.show()


***Taken other models for comparsion***

In [ ]:
import time
from Crypto.Cipher import AES, DES
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Util.Padding import pad, unpad
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes
from Crypto.Random import get_random_bytes

# Sample plaintext
plaintext = b"This is a secret message that must be encrypted!!"  # Length must be multiple of 16

# Helper function to measure execution time
def measure_time(func, *args):
    start = time.time()
    result = func(*args)
    end = time.time()
    return result, end - start

### --- AES-256 Encryption & Decryption --- ###
def aes_encrypt(plaintext, key):
    cipher = AES.new(key, AES.MODE_ECB)
    padded_text = pad(plaintext, AES.block_size)
    return cipher.encrypt(padded_text)

def aes_decrypt(ciphertext, key):
    cipher = AES.new(key, AES.MODE_ECB)
    decrypted_padded_text = cipher.decrypt(ciphertext)
    return unpad(decrypted_padded_text, AES.block_size)

### --- Hybrid AES-128 & DES Encryption --- ###
def hybrid_encrypt(plaintext, aes_key, des_key):
    cipher_aes = AES.new(aes_key, AES.MODE_ECB)
    aes_output = cipher_aes.encrypt(pad(plaintext, AES.block_size))
    cipher_des = DES.new(des_key, DES.MODE_ECB)
    return cipher_des.encrypt(aes_output)

def hybrid_decrypt(ciphertext, aes_key, des_key):
    cipher_des = DES.new(des_key, DES.MODE_ECB)
    des_output = cipher_des.decrypt(ciphertext)
    cipher_aes = AES.new(aes_key, AES.MODE_ECB)
    return unpad(cipher_aes.decrypt(des_output), AES.block_size)

### --- RSA Encryption & Decryption --- ###
def rsa_encrypt(plaintext, rsa_key):
    cipher = PKCS1_OAEP.new(rsa_key)
    return cipher.encrypt(plaintext)

def rsa_decrypt(ciphertext, rsa_key):
    cipher = PKCS1_OAEP.new(rsa_key)
    return cipher.decrypt(ciphertext)

### --- ECC Key Exchange (Instead of Direct Encryption) --- ###
def ecc_generate_keys():
    private_key = ec.generate_private_key(ec.SECP256R1())
    public_key = private_key.public_key()
    return private_key, public_key

def ecc_encrypt(plaintext, public_key):
    digest = hashes.Hash(hashes.SHA256())
    digest.update(plaintext)
    return digest.finalize()

def ecc_decrypt(ciphertext, private_key):
    return ciphertext

### --- Benchmarking --- ###
aes_key = get_random_bytes(32)
aes_key_128 = aes_key[:16]
des_key = get_random_bytes(8)
rsa_key = RSA.generate(2048)
rsa_public = rsa_key.publickey()
ecc_private, ecc_public = ecc_generate_keys()

aes_enc, aes_enc_time = measure_time(aes_encrypt, plaintext, aes_key)
aes_dec, aes_dec_time = measure_time(aes_decrypt, aes_enc, aes_key)

hybrid_enc, hybrid_enc_time = measure_time(hybrid_encrypt, plaintext, aes_key_128, des_key)
hybrid_dec, hybrid_dec_time = measure_time(hybrid_decrypt, hybrid_enc, aes_key_128, des_key)

rsa_enc, rsa_enc_time = measure_time(rsa_encrypt, plaintext, rsa_public)
rsa_dec, rsa_dec_time = measure_time(rsa_decrypt, rsa_enc, rsa_key)

ecc_enc, ecc_enc_time = measure_time(ecc_encrypt, plaintext, ecc_public)
ecc_dec, ecc_dec_time = measure_time(ecc_decrypt, ecc_enc, ecc_private)

### --- Results --- ###
print("\n=== Performance vs Security Trade-off ===")
print(f"AES-256 Encryption Time: {aes_enc_time:.6f} sec | Decryption Time: {aes_dec_time:.6f} sec")
print(f"Hybrid AES-DES Encryption Time: {hybrid_enc_time:.6f} sec | Decryption Time: {hybrid_dec_time:.6f} sec")
print(f"RSA Encryption Time: {rsa_enc_time:.6f} sec | Decryption Time: {rsa_dec_time:.6f} sec")
print(f"ECC Hashing Time: {ecc_enc_time:.6f} sec | Simulated Decryption Time: {ecc_dec_time:.6f} sec")


***Code for graph***

In [ ]:
import matplotlib.pyplot as plt

# Data for different encryption models
models = ['AES-256', 'Hybrid AES-DES', 'RSA', 'ECC Hashing', 'Proposed Model']

# Encryption time (in seconds)
encryption_times = [0.000115, 0.000227, 0.001274, 0.004918, 0.00031]
decryption_times = [0.000097, 0.000074, 0.002178, 0.000001, 0.00025]
data_expansion = [130.61, 130.61, 522.45, 65.31, 24.00]

# Plot Encryption Time
plt.figure(figsize=(6,4))
plt.bar(models, encryption_times, color=['blue', 'red', 'green', 'purple', 'orange'])
plt.title('Encryption Time Comparison')
plt.ylabel('Time (seconds)')
plt.xticks(rotation=30, ha='right')
plt.show()

# Plot Decryption Time
plt.figure(figsize=(6,4))
plt.bar(models, decryption_times, color=['blue', 'red', 'green', 'purple', 'orange'])
plt.title('Decryption Time Comparison')
plt.ylabel('Time (seconds)')
plt.xticks(rotation=30, ha='right')
plt.show()

# Plot Data Expansion
plt.figure(figsize=(6,4))
plt.bar(models, data_expansion, color=['blue', 'red', 'green', 'purple', 'orange'])
plt.title('Data Expansion Comparison')
plt.ylabel('Expansion (%)')
plt.xticks(rotation=30, ha='right')
plt.show()
